# Setup Code (Run first)

In [ ]:
# Capture the output but do not do anything with it
# Just to avoid cluttering the output

%pip install - -upgrade git+https: // github.com/terrierteam/pyterrier_doc2query.git
%pip install python-terrier == 0.9.2
%pip install semanticscholar
%pip install gensim
%pip install seaborn

# Load all packages and initialize pyTerrier
from gensim.models import Word2Vec
from semanticscholar import SemanticScholar
import numpy as np
import pandas as pd
import pyterrier as pt
import os

if not pt.started():
    pt.init()

# Prepares the index path and avoid errors with already existing indices
index_count = 0

In [63]:
# Helper function to intialize multiple indices
def prepare_index_path(indexName):
    global index_count
    index_count = index_count + 1
    index_path = indexName + str(index_count)

    if os.path.exists(index_path) & os.path.isdir(index_path):
        files = os.listdir(index_path)
        for file in files:
            file_name = index_path + '/' + file
            os.remove(file_name)
        os.rmdir(index_path)
    elif os.path.exists(index_path) & (not os.path.isdir(index_path)):
        os.rmove(index_path)

    return os.path.abspath(index_path)

def build_index(indexName, dataset):
    index_path = prepare_index_path(indexName)
    indexer = pt.IterDictIndexer(
        index_path, overwrite=True, blocks=True)
    indexer.setProperty(
        "stopwords.filename", "F:\Bibliotheken\Desktop\Skripte\packgaabwir2022\en.txt")
    index_created = indexer.index(dataset.get_corpus_iter(),
                            fields=['title', 'doi', 'abstract'],
                            meta=('docno',))
    return index_created

In [62]:
index_path = 'treccovid' + str(2)
print(index_path)
print(os.path.exists(index_path))
print(os.path.isdir(index_path))
files = os.listdir(index_path)
norm_path = os.path.normpath(index_path)
abs_path = os.path.abspath(norm_path)
print(norm_path)
print(abs_path)

treccovid2
True
True
treccovid2
f:\Bibliotheken\Desktop\Skripte\packgaabwir2022\treccovid2


# Vorgehen

## Allgemein

TREC-Covid-Queries verwenden und erweitern um weitere Anfrageterme, um die Retrieval-Performance zu verbessern (Query Expansion). 

Für die QE müssen Termkandidaten ("set of C" c_1, c_2, c_3, ...) bestimmt werden, die anschließend gerankt werden.
D.h. ihr sendet zunächst die Standard-Query ab und später nochmal für die finale Evaluierung, die Query mit Termerweiterungen.

## QE 
Wenn wenig Zeit:
Nur die globale Variante evaluieren, wobei ja der "Claim for Fame" im Paper ist, dass die lokale Methode bessere Ergebnisse liefert.
### LOKALE QE 
Word2Vec-Embeddings auf TREC-Covid trainieren (https://radimrehurek.com/gensim/)


In [ ]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
docs = pd.DataFrame(dataset.get_corpus_iter())
abstracts = list(docs["abstract"])
model = Word2Vec(sentences=abstracts,window=10,sg=1,seed=1)
model.save("word2vec_titles.model")

In [ ]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
docs = pd.DataFrame(dataset.get_corpus_iter())
titles = list(docs["title"])
model = Word2Vec(sentences=titles,window=10,sg=1,seed=1)
model.save("word2vec_titles.model")

In [ ]:
model.wv.key_to_index

In [13]:
from sklearn.decomposition import PCA
from matplotlib import pyplot
# fit a 2d PCA model to the vectors
X = model[model.wv.key_to_index]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.key_to_index)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

TypeError: 'Word2Vec' object is not subscriptable

In [ ]:

dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
trec_covid_index_ref = build_index('trecCovid',dataset)


In [70]:
trec_covid_index = pt.IndexFactory.of(trec_covid_index)
tokens = []
for kv in trec_covid_index.getLexicon():
    tokens.append([kv.getKey(),	kv.getValue().getFrequency()])

cord19/trec-covid documents:   0%|          | 0/192509 [19:22<?, ?it/s]


In [71]:
tokens

[['0', 109999],
 ['00', 1735],
 ['000', 6468],
 ['0001', 2587],
 ['0002', 195],
 ['0003', 146],
 ['0004', 134],
 ['0005', 167],
 ['0006', 102],
 ['0007', 100],
 ['0008', 65],
 ['0009', 73],
 ['000copi', 2],
 ['000d', 9],
 ['000da', 1],
 ['000h', 3],
 ['000kwh', 2],
 ['000m', 1],
 ['000mt', 1],
 ['000ng', 1],
 ['000pc', 1],
 ['000pg', 1],
 ['000x', 1],
 ['000year', 1],
 ['001', 12534],
 ['0010', 30],
 ['0010a', 1],
 ['0011', 29],
 ['0012', 47],
 ['0013', 48],
 ['0014', 81],
 ['0015', 49],
 ['0016', 58],
 ['0017', 26],
 ['0018', 34],
 ['0019', 48],
 ['001uncorr', 1],
 ['002', 2685],
 ['0020', 34],
 ['0021', 48],
 ['0022', 74],
 ['0023', 22],
 ['0024', 30],
 ['0025', 27],
 ['0026', 31],
 ['0027', 27],
 ['0028', 63],
 ['0029', 40],
 ['002c', 1],
 ['002vs0', 1],
 ['003', 2313],
 ['0030', 36],
 ['0031', 54],
 ['0032', 46],
 ['0033', 54],
 ['0034', 98],
 ['0035', 62],
 ['0036', 60],
 ['0037', 69],
 ['0037a', 1],
 ['0038', 42],
 ['0039', 40],
 ['003r', 1],
 ['004', 2003],
 ['0040', 159],
 ['00

### GLOBALE QE
Standardvariante wie z.B. auf Basis von Wikipedia.

# 1. Standard-Query an den Index für erstes Ranking
Aus dem Title des Topics, quasi wie in der Standard-Pipeline in Pyterrier.
Paper Inverse Document Frequency model (InL2)
Zuerst BM25.

# 2. Erstellung der Termkandidaten
## 2.1 Top-3 Dokumente
Alle Terme (ausschließlich der Stoppworte).

## 2.2 Terme der Referenzen
Alle Terme aus den Referenzen der 3 Dokumente (Terme der Titel).

## 2.3 Erweiterung mit Co-Autoren
Über die Koautoren können noch weitere Dokumente bzw. die entsprechenden Terme hinzugefügt werden. Aus Zeitgründen könnte man vielleicht auch diesen Schritt weglassen, wenn es zu viel Aufwand ist über Koautoren weitere Dokumente zu finden. Bei der Implementierung scheint ihr ja aber schon recht weit zu sein. Die "relevanten Paper der Autoren" sind einfach die Top-k Dokumente oder möglicherweise alle zusätzlichen Papers, die über die Koautorenschaft gefunden werden, gemeint. Daher ergibt sich auf der Name PSEUDO-Relevanz-Feedback, da einfach angenommen wird, dass die Top-Treffer alle relevant sind.

# 2.4 Ranking aller Terms
Paper: Bo1
Top-k Terme auswählen (k selbst wählen).
Ergebnis: Potentielle Kandidaten für eine Termwerweiterung

# 2.5 Word2Vec-Modell mit Terms nachtrainieren
Über die Kosinusähnlichkeit der Embeddings der ursprünglichen Anfrageterme bestimmt ihr nun weitere Terme aus dem Word2Vec-Modell. Diese Termkandidaten werden dann wie in 2.1.4 über Bo1 o.ä. gerankt.

In [ ]:
mv_model = Word2Vec.load("data/word2vec.model")

queries = preprocess_queries(queries, augmented=True, mv_model=mv_model)
display(queries.head(5))

pq_title = queries[['query_id', 'title']]
pq_title = pq_title.rename(columns={'query_id':'qid', 'title':'query'})
display(pq_title.head(5))

pq_desc = queries[['query_id', 'description']]
pq_desc = pq_desc.rename(columns={'query_id':'qid', 'description':'query'})
display(pq_desc.head(5))

pq_narr = queries[['query_id', 'narrative']]
pq_narr = pq_narr.rename(columns={'query_id':'qid', 'narrative':'query'})
display(pq_narr.head(5))

# 2.6 Zwei Sets zusammenfügen und Top-k für QE der ursprünglichen Querry nutzen
Diese jeweils für die 50 Topics absenden.

Ergebnis: Die finalen Rankings, die dann ausgewertet werden können.